In [0]:
%sql
CREATE DATABASE `retail-sales-project`.db;

In [0]:
%sql
CREATE VOLUME `retail-sales-project`.db.volume;

In [0]:
raw_customers=spark.read.format("json").load("/Volumes/retail-sales-project/db/volume/Bronze/customer_dataset.json")
display(raw_customers.head(5))

age,city,customer_id,gender,loyalty_tier,signup_date
35.0,Delhi,C001,M,Gold,01-Apr-2020
39.0,Gurgaon,C002,null,silver,14/11/2023
null,Kolkata,C003,F,Silver,2022/09/13
null,BENGALURU,C004,Male,null,2017-02-21
19.0,Pune,C005,Male,PLATINUM,2022/06/06


In [0]:
raw_orders=spark.read.csv("/Volumes/retail-sales-project/db/volume/Bronze/retail_dataset.csv",header=True)
display(raw_orders.limit(5))

order_id,order_date,customer_id,customer_name,product_id,product_name,category,quantity,price,payment_type,order_status,returned
1001,2025-07-28,C004,Customer_4,P103,Denim Jean,Apparel,1,339.97,card,shipped,no
1002,2025-09-09,C054,Customer_54,P117,USB Cable,Accessories,1,$380.61,cash,completed,no
1003,2025-08-13,C059,Customer_59,P101,T-shirt XL,Apparel,3,278.26,card,completed,no
1004,11-Sep-2025,C013,Customer_13,P107,Charger,Accessories,3,$85.51,upi,completed,no
1005,24/08/2025,C035,Customer_35,P112,Backpack,Accessories,1,343.88,cash,cancelled,no


In [0]:
from pyspark.sql.functions import *
spark.conf.set("spark.sql.ansi.enabled","false")

order_date=coalesce(
    to_timestamp(trim(col("order_date")), "dd/MM/yyyy"),
    to_timestamp(trim(col("order_date")), "dd-MM-yyyy"),
    to_timestamp(trim(col("order_date")), "yyyy-MM-dd"),
    to_timestamp(trim(col("order_date")), "d/MMM/yyyy"),
    to_timestamp(trim(col("order_date")), "MM/dd/yyyy"),
    to_timestamp(trim(col("order_date")), "dd-MMM-yyyy"),
    to_timestamp(trim(col("order_date")), "MM-dd-yyyy")
)

clean_orders=(raw_orders
              .withColumn("order_id",trim(col("order_id")).cast("long"))
              .withColumn("order_date",order_date)
              .withColumn("customer_id",trim(col("customer_id")))
              .withColumn("customer_name",trim(col("customer_name")))
              .withColumn("product_id",trim(col("product_id")))
              .withColumn("product_name",lower(trim(col("product_name"))))
              .withColumn("category",lower(trim(col("category"))))
              .withColumn("quantity",when(trim(col("quantity")).isNull() | (trim(col("quantity"))==""),lit(1)).otherwise(trim(col("quantity"))))
              .withColumn("quantity",col("quantity").cast("int"))
              .withColumn("quantity",when(col("quantity")<=0,lit(1)).otherwise(col("quantity")))
              .withColumn("price",translate(col("price"),"$","").cast("double"))
              .withColumn("payment_type",lower(trim(col("payment_type"))))
              .withColumn("order_status",lower(trim(col("order_status"))))
              .withColumn("returned",lower(trim(col("returned"))))
              .withColumn("total_price",(coalesce(col("quantity"),lit(0))*coalesce(col("price"),lit(0.0))).cast("double")) 
              
              
)

clean_orders=clean_orders.dropDuplicates(["order_id","product_id"]).filter(col("order_id").isNotNull()&col("product_id").isNotNull())

# display(clean_orders.limit(10))

clean_orders.write.format("delta").mode("overwrite").saveAsTable("`retail-sales-project`.db.silver_orders")

In [0]:
%sql
select * from `retail-sales-project`.db.silver_orders;


order_id,order_date,customer_id,customer_name,product_id,product_name,category,quantity,price,payment_type,order_status,returned,total_price
1004,2025-09-11T00:00:00.000Z,C013,Customer_13,P107,charger,accessories,3,85.51,upi,completed,no,256.53000000000003
1133,2025-08-03T00:00:00.000Z,C032,Customer_32,P101,t-shirt xl,apparel,1,378.42,upi,returned,yes,378.42
1036,2025-02-08T00:00:00.000Z,C061,Customer_61,P104,headphone,electronics,3,351.24,upi,completed,no,1053.72
1034,2025-08-31T00:00:00.000Z,C004,Customer_4,P117,usb cable,accessories,2,46.55,cash,delivered,no,93.1
1154,2025-07-24T00:00:00.000Z,C057,Customer_57,P106,smartphone,electronics,1,338.12,upi,cancelled,no,338.12
1213,2025-09-10T00:00:00.000Z,C063,Customer_63,P113,belt,accessories,1,260.06,upi,returned,yes,260.06
1093,2025-08-24T00:00:00.000Z,C003,Customer_3,P102,sneakers,shoes,2,60.04,upi,returned,yes,120.08
1083,2025-08-07T00:00:00.000Z,C058,Customer_58,P114,adapter,accessories,2,324.11,cash,returned,yes,648.22
1276,2025-08-22T00:00:00.000Z,C025,Customer_25,P109,jeans,apparel,1,309.52,card,delivered,no,309.52
1138,2025-09-07T00:00:00.000Z,C051,Customer_51,P106,smartphone,electronics,3,193.43,cash,cancelled,no,580.29


In [0]:
display(raw_customers.limit(8))

age,city,customer_id,gender,loyalty_tier,signup_date
35.0,Delhi,C001,M,Gold,01-Apr-2020
39.0,Gurgaon,C002,null,silver,14/11/2023
null,Kolkata,C003,F,Silver,2022/09/13
null,BENGALURU,C004,Male,null,2017-02-21
19.0,Pune,C005,Male,PLATINUM,2022/06/06
37.0,Hyderabad,C006,f,null,27-Apr-2018
50.0,Kolkata,C007,M,null,2021/11/02
59.0,Bangalore,C008,null,Silver,17-Jun-2021


In [0]:
from pyspark.sql.functions import *
spark.conf.set("spark.sql.ansi.enabled","false")


date=coalesce(
    to_timestamp(trim(col("signup_date")), "dd/MM/yyyy"),
    to_timestamp(trim(col("signup_date")), "dd-MM-yyyy"),
    to_timestamp(trim(col("signup_date")), "yyyy-MM-dd"),
    to_timestamp(trim(col("signup_date")), "d/MMM/yyyy"),
    to_timestamp(trim(col("signup_date")), "MM/dd/yyyy"),
    to_timestamp(trim(col("signup_date")), "dd-MMM-yyyy"),
    to_timestamp(trim(col("signup_date")), "MM-dd-yyyy")
)


clean_customers=(
    raw_customers
            .withColumn("customer_id",trim(col("customer_id")))
            .withColumn("gender",lower(trim(col("gender"))))
            .withColumn("age",when (trim(col("age")).isNull() | (trim(col("age"))==""),None).otherwise(col("age")))
            .withColumn("age",when(col("age").cast("int")<0,None).otherwise(col("age").cast("int")))
            .withColumn("city",lower(trim(col("city"))))
            .withColumn("loyalty_tier",lower(trim(col("loyalty_tier"))))
            .withColumn("signup_date",date)
)


display(clean_customers.limit(8))

clean_customers.write.format("delta").mode("overwrite").saveAsTable("`retail-sales-project`.db.silver_customers")

age,city,customer_id,gender,loyalty_tier,signup_date
35,delhi,C001,m,gold,2020-04-01T00:00:00.000Z
39,gurgaon,C002,null,silver,2023-11-14T00:00:00.000Z
null,kolkata,C003,f,silver,null
null,bengaluru,C004,male,null,2017-02-21T00:00:00.000Z
19,pune,C005,male,platinum,null
37,hyderabad,C006,f,null,2018-04-27T00:00:00.000Z
50,kolkata,C007,m,null,null
59,bangalore,C008,null,silver,2021-06-17T00:00:00.000Z


# **GOLD LAYER**

In [0]:
display(orders_enriched.head(5))

customer_id,order_id,order_date,customer_name,product_id,product_name,category,quantity,price,payment_type,order_status,returned,total_price,age,city,gender,loyalty_tier,signup_date,year,month
C013,1004,2025-09-11,Customer_13,P107,charger,accessories,3,85.51,upi,completed,no,256.53000000000003,34,null,m,silver,2023-09-06T00:00:00.000Z,2025,9
C032,1133,2025-08-03,Customer_32,P101,t-shirt xl,apparel,1,378.42,upi,returned,yes,378.42,null,pune,null,silver,2018-12-23T00:00:00.000Z,2025,8
C061,1036,2025-02-08,Customer_61,P104,headphone,electronics,3,351.24,upi,completed,no,1053.72,65,gurgaon,male,bronze,2020-06-22T00:00:00.000Z,2025,2
C004,1034,2025-08-31,Customer_4,P117,usb cable,accessories,2,46.55,cash,delivered,no,93.1,null,bengaluru,male,null,2017-02-21T00:00:00.000Z,2025,8
C057,1154,2025-07-24,Customer_57,P106,smartphone,electronics,1,338.12,upi,cancelled,no,338.12,26,gurgaon,male,gold,2022-10-12T00:00:00.000Z,2025,7


In [0]:
from pyspark.sql.functions import *
spark.conf.set("spark.sql.ansi.enabled","false")

silver_orders=spark.table("`retail-sales-project`.db.silver_orders")
silver_customers=spark.table("`retail-sales-project`.db.silver_customers")

orders_enriched=silver_orders.join(silver_customers,on='customer_id',how='left')

# display(orders_enriched.head(5))

orders_enriched=(orders_enriched
                .withColumn("order_date",to_date(col("order_date")))
                .withColumn("year",year(col("order_date")))
                .withColumn("month",month(col("order_date")))
)

# display(orders_enriched.head(5))

gold_df = (
    orders_enriched
    .groupBy("product_id","product_name","customer_id","category","year","month","order_date","gender","age","city","loyalty_tier")
    .agg(
        sum("total_price").alias("total_sales"),
        sum(coalesce(col("quantity"), lit(0))).alias("total_quantity"),
        countDistinct("order_id").alias("total_orders"),
        min("price").alias("min_price"),
        max("price").alias("max_price"),
        avg("price").alias("avg_unit_price"),
        sum(when(col("returned") == "yes", 1).otherwise(0)).alias("returned_count"),
        sum(when(col("returned") == "yes", col("price")).otherwise(0.0)).alias("returned_amount")
    )
)

# display(gold_df.limit(6))

gold_df = gold_df.withColumn("avg_order_value", when(col("total_orders")>0, col("total_sales")/col("total_orders")).otherwise(lit(0.0))) \
                 .withColumn("avg_price_per_item", when(col("total_quantity")>0, col("total_sales")/col("total_quantity")).otherwise(lit(0.0))) \
                 .withColumn("refreshed_at", current_timestamp())


# display(gold_df.head(5))

gold_df.write.format("delta").mode("overwrite").saveAsTable("`retail-sales-project`.db.gold_aggregates")